In [1]:
####################################################################################################################
#################################################Import#############################################################
####################################################################################################################
import timeit
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import copy
from matplotlib.collections import LineCollection
import sys
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
# np.random.seed(seed=125)
np.random.seed(seed=2806)




In [2]:
####################################################################################################################
#################################################Functions Definition###############################################
####################################################################################################################

# Function that separate the episode data into features and label
def data_separate (dataset,train):
    global X
    global y    
    if train ==1:
        X = dataset.iloc[:,0:dataset.shape[1]-1]  # all rows, all the features and no labels
        y = dataset.iloc[:, dataset.shape[1]-2:dataset.shape[1]]  # all rows, label and batch_index only
    else:
        X = dataset.iloc[:,0:dataset.shape[1]-1]  # all rows, all the features and no labels
        y = dataset.iloc[:, dataset.shape[1]-1:dataset.shape[1]]  # all rows, label and batch_index only  
    return X,y


# Function that chooses exploration or explotation method
def exploration_explotation(epsilon):
    global exploration 
    if np.random.rand() < epsilon:  
        exploration=1
    else:
        exploration=0    
    return exploration


# Function that returns all available actions in the state given as an argument: 
def available_actions(number_of_columns,batches,initial_state,current_state,trashold, exploration):
    global exclude
    global all_batches
#    exclude=[]
    all_batches=np.arange(number_of_batches+1)
    # remove columns that have been already selected
    exclude=copy.copy(batches)
    # remove the initial_state and the current_state
    exclude.extend([initial_state, current_state])
    available_act = list(set(all_batches)-set(exclude))
    # remove actions that have negetiv Q value
    if exploration==0:
        index = np.where(Q[current_state,available_act] > trashold)[1]
        available_act= [available_act[i] for i in index.tolist()]
    return available_act

# Function that chooses which action to be performed according to exploration - explotation method
def sample_next_action(current_state, Q, available_act, exploration):
    global available_act_q_value
    available_act_q_value=np.array(Q[current_state,available_act]).reshape(-1,).tolist()
    if exploration==1: 
        #random selection
        next_action = int(np.random.choice(available_act,1))
    else: 
        #greedy selection according to max value 
        maxQ=max(available_act_q_value)
        count = available_act_q_value.count(maxQ)
        if count > 1:
            max_columns =[i for i in range(len(available_act_q_value)) if available_act_q_value[i] == maxQ]
            i = np.random.choice(max_columns)
        else:
            i = available_act_q_value.index(maxQ)
        next_action=available_act[i]        
    return next_action

# function that update a list with all selected columns in the episode
def update_batches(action, batches):
    update_batches=batches
    update_batches.append(action)
    return update_batches

# function that update the X_train according to the current episode batches list 
def update_X_Y_train(batches,X_train_main,Y_train_main):
    X_train=X_train_main[X_train_main['Batch_Index'].isin(batches)]
    X_train=pd.DataFrame(X_train)
    Y_train=Y_train_main[Y_train_main['Batch_Index'].isin(batches)]
    Y_train=pd.DataFrame(Y_train)
    return X_train,Y_train

# Function that run the learner and get the error to the current episode columns list
def Learner(X_train, X_test,y_train, y_test):
    global learner
    global y_pred
    if learner_model == 'DT':
        learner = tree.DecisionTreeClassifier(random_state=0)
        learner = learner.fit(X_train, y_train)   #remove batch_index column
        y_pred = learner.predict(X_test)
    elif learner_model == 'KNN':
        learner = KNeighborsClassifier(metric='hamming',n_neighbors=5)
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)        
    elif learner_model == 'SVM':
        learner = SVC()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)        
    elif learner_model == 'NB':
        learner = MultinomialNB()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)
    elif learner_model == 'AB':
        learner = AdaBoostClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    elif learner_model == 'GB':
        learner = GradientBoostingClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    accuracy=metrics.accuracy_score(y_test, y_pred)
    error=1-accuracy
    return error

# Function that updates the Q matrix according to the path selected and the Q 
def q_update(current_state ,action,learning_rate, reward):
    # next_state = current action
    max_index = np.where(Q[action,] == np.max(Q[action,]))[1]    
    if max_index.shape[0] > 1:
        #np.random.seed(seed)
        max_index = int(np.random.choice(max_index, size = 1))
    else:
        max_index = int(max_index)
    max_value = Q[action, max_index]
    #we start with 1 for all Q values and update with reward only at the 1st time
    if Q[current_state, action]==1:
        Q[current_state, action] = learning_rate*reward
    else:
        Q[current_state, action] = Q[current_state, action]+ learning_rate*((reward + (discount_factor * max_value))-Q[current_state, action])
    td_error = abs((reward + (discount_factor * max_value))-Q[current_state, action])
    return td_error
    #Q[current_state, action] = Q[current_state, action]+ learning_rate*(reward + (discount_factor * max_value)-Q[current_state, action])
    
#split to n batches - change to K-menas later?   
def batch_split(data, n,split_type):
    if split_type == 1: #random split
        chunk = 0
        n = int(round(len(data)/number_of_batches))
        for i in xrange(0, len(data), n):
            data.iloc[i:i + n,data.columns.get_loc('Batch_Index')] = chunk
            chunk+=1
        return  data.Batch_Index   
    else: # Kmenas
        kmeans = KMeans(n_clusters=n, random_state=0).fit(data.iloc[:,0:data.shape[1]-2])
        return kmeans.labels_

In [3]:
####################################################################################################################
#################################################Parameters#########################################################
####################################################################################################################

#Experiment parameters: 
location = 'Processed_Datasets'
file='covtype' #Wine #mushrooms #Normal #Letter #Iris #glass

train_data='_Train_Data.csv'
# test_data='_Test_Data.csv'
val_data='_Val_Data.csv'

experiment= file +'_ver2'

number_of_experiment=1


# Q learning parameter # 
learning_rate=0.005
discount_factor=0.01
epsilon = 0.1

# Learner and episode parameters #
learner_model = 'DT' #DT #KNN #SVM
episode_size=100
internal_trashold=0
external_trashold=0


sampling_rate=0.5

#Experiments folder management: 
if not os.path.exists('Experiments/'+ str(experiment)):
    os.makedirs('Experiments/'+ str(experiment))
else:
    shutil.rmtree('Experiments/'+ str(experiment))        
    os.makedirs('Experiments/'+ str(experiment))
writer = pd.ExcelWriter('Experiments/'+ str(experiment) + '/df.xlsx') 


In [4]:
data = pd.read_csv(location + '/' + file + train_data)
# data_test = pd.read_csv(location + '/' + file + test_data, sep='\t')
data_val = pd.read_csv(location + '/' + file + val_data)
del data_val['Batch_Index']
print (data.head(2))
print (data_val.head(2))

number_of_batches =  len(data.Batch_Index.unique())
print(number_of_batches)
number_of_episodes = number_of_batches*100
print(number_of_episodes)


   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0       2811     337     14                               201   
1       3050       0      2                               450   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                              79                             3296   
1                             105                             2818   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0            189             220            170   
1            216             234            156   

   Horizontal_Distance_To_Fire_Points  ...  Soil_Type33  Soil_Type34  \
0                                6126  ...            0            0   
1                                1475  ...            0            0   

   Soil_Type35  Soil_Type36  Soil_Type37  Soil_Type38  Soil_Type39  \
0            0            0            0            0            0   
1            0            0            0            0            0   

   Soil_Typ

In [5]:
# epsilon=0.1
# learning_rate=0.005

In [ ]:
####################################################################################################################
#################################################Experimrnt Code####################################################
####################################################################################################################

for e in range (number_of_experiment):
    print ('Experiments ' + str(e) + ' start')
##########################Experiment setup##########################
    start = timeit.default_timer()
    print ("Number of episodes: "+ str(number_of_episodes)) 
    # Initialize matrix Q as a 1 values matrix:
    Q = np.matrix(np.ones([number_of_batches+1,number_of_batches+1])) # we will use the last dummy columns as initial state s
    # Set initial_state to be the last dummy column we have created
    initial_state=number_of_batches
    # define data frame to save episode policies results
    df = pd.DataFrame(columns=('episode','episode_batches','policy_batches','policy_accuracy_val'))
    print ("initial state number: "+ str(initial_state) + " (the last dummy column we have created)") 
    #Separate test data
    X_test_all ,y_test_all =data_separate(data_val,0)
    td_error_mega_list = []
    sliding_error_list = []
    

    
    ##########################  episode  ##########################  
    for i in range (int(number_of_episodes)):
    ########## Begining of episode  ############
        # Initiate lists for available_act, episode_columns and and the policy mode & episode_error
        episode_available_act=list(np.arange(number_of_batches))
        td_error_list = []
        episode_batches=[]
        policy=0
        episode_error=0
        # Initiate the error to 0.5
        episode_last_error=0.5
        # Initiate current_state to be initial_state
        episode_current_state=initial_state
        # Create the episode data 
#         episode= data.copy()
        episode_data = data.groupby('Batch_Index', group_keys=False).apply(pd.DataFrame.sample, frac=sampling_rate)
        episode_train, episode_test = train_test_split(episode_data, test_size=0.2)
#         episode_train = data.copy() 
#         episode_test = data_val.copy()
        
        # Separate the episode data into features and label
        X_train_episode_all,y_train_episode_all=data_separate(episode_train,1)
        X_test_episode ,y_test_episode =data_separate(episode_test ,0)
        #X_train_main_episode, X_test_main_episode, y_train_episode, y_test_episode = data_split(X_episode,y_episode)
        if i<number_of_episodes*0.25:
            epsilon=0.9 
            learning_rate=0.09
        elif i<number_of_episodes*0.5:
            epsilon=0.5
            learning_rate=0.05
        elif i<number_of_episodes*0.85:
            epsilon=0.4
            learning_rate=0.01
        else:
            epsilon=0.1
            learning_rate=0.005
        ########## Q learning start ############

        while len(episode_available_act)>0:
            # Get exploration or explotation flag 
            exploration=exploration_explotation(epsilon)
            # Get available actions in the current state
            episode_available_act = available_actions(number_of_batches,episode_batches,initial_state,episode_current_state,internal_trashold,exploration)
            if len(episode_available_act)>0:
                # Sample next action to be performed
                episode_action = sample_next_action(episode_current_state, Q, episode_available_act, exploration)
                # Update the episode_batches
                episode_batches=update_batches(episode_action,episode_batches)
                # Update the dataset to include all episode batches + current selected action (batches)
                X_train_episode, y_train_episode =update_X_Y_train(episode_batches,X_train_episode_all,y_train_episode_all)
                
                if len(X_train_episode)>0:
                # Update the accuracy of the current batches
                    episode_error= Learner(X_train_episode.iloc[:,0:X_train_episode.shape[1]-1], 
                                           X_test_episode.iloc[:,0:X_test_episode.shape[1]-1] , 
                                           y_train_episode.iloc[:, -1], 
                                           y_test_episode.iloc[:, -1])
                    # Update reward
                    episode_reward=episode_last_error-episode_error
                    # Update Q matrix
                    td_error = q_update(episode_current_state,episode_action,learning_rate, episode_reward)
                    td_error_list.append(td_error)

                    # Update parameters for next round 
                    episode_current_state=episode_action
                    episode_last_error=episode_error
        ########## Q learning End ############
        # Calculate policy 
        policy_available_actions=list(np.arange(number_of_batches))
        policy_batches=[]
        policy_current_state=initial_state
        while len(policy_available_actions)>0:
            # Get available actions in the current state
            policy_available_actions = available_actions(number_of_batches,policy_batches,initial_state,policy_current_state, external_trashold, exploration=0)
            # # Sample next action to be performed
            if len(policy_available_actions)>0:
                policy_select_action = sample_next_action(policy_current_state, Q, policy_available_actions, exploration=0)
                # Update the episode_columns
                policy_batches=update_batches(policy_select_action,policy_batches)
                policy_current_state=policy_select_action
        # Calculate policy_accuracy    
        if len(policy_batches)>0:
            ##for training dataset##
            X_train_policy, Y_train_policy =update_X_Y_train(policy_batches, X_train_episode_all, y_train_episode_all)
            policy_error=Learner(X_train_policy.iloc[:,0:X_train_policy.shape[1]-1], 
                                 X_test_all, Y_train_policy.iloc[:, -1], y_test_all)
            policy_accuracy_test=1-policy_error
        else:
            policy_accuracy_test=0 
        df=df.append({'episode':str(i+1), 'episode_batches':str(episode_batches),'policy_batches':str(policy_batches),'policy_accuracy_test':policy_accuracy_test}, ignore_index=True)
        #Prints
        print ("episode "+ str(i+1) +" start") 
        print ("episode batches: "+ str(episode_batches) + " epsilon: " + str(epsilon) + " learning rate: " + str(learning_rate) + " error: " +str(episode_error))
        print ("episode policy:" + str(policy_batches) + " accuracy: " + str(policy_accuracy_test)) 
        #print ("Q Matrix" + str(Q))
        print ("episode "+ str(i+1) +" end") 
        avg_td_error = np.mean(td_error_list)
        td_error_mega_list.append(avg_td_error)
        sliding_error = np.mean(td_error_list[-50:])
        sliding_error_list.append(sliding_error)
        print ("sliding td error: {}".format(sliding_error))
    ########## End of episode  ###############
    df.to_excel(writer, 'Experiment' + str(e))
    df_plot=df[['episode','policy_accuracy_test']]
    plot=df_plot.plot()
    plt.plot(sliding_error_list)
    fig = plot.get_figure()
    fig.savefig('Experiments/'+ str(experiment) + '/plot_experiment_' + str(e) +'.png')
    #restart kernel
    stop = timeit.default_timer()
    duration = stop - start
    print ("run time")
    print(duration)
    pd.DataFrame(Q).to_csv('Experiments_'+ str(experiment)+ '_Q.csv') 
writer.save()


Experiments 0 start
Number of episodes: 43000
initial state number: 430 (the last dummy column we have created)
episode 1 start
episode batches: [394, 283, 77, 221, 325, 152, 206, 322, 417, 264, 330, 125, 24, 268, 338, 72, 189, 138, 358, 10, 134, 395, 79, 279, 316, 35, 222, 131, 38, 425, 410, 343, 327, 171, 122, 227, 357, 58, 270, 292, 415, 89, 391, 167, 106, 367, 359, 75, 76, 200, 261, 266, 352, 317, 190, 212, 176, 61, 411, 360, 345, 123, 386, 371, 191, 94, 161, 37, 249, 287, 253, 43, 33, 151, 400, 239, 406, 34, 92, 88, 40, 45, 170, 177, 290, 196, 31, 375, 305, 139, 187, 28, 429, 137, 326, 346, 136, 55, 3, 248, 83, 142, 218, 389, 36, 252, 175, 369, 56, 165, 209, 182, 208, 128, 240, 272, 23, 424, 27, 164, 349, 254, 277, 201, 5, 408, 154, 127, 216, 312, 250, 9, 246, 32, 241, 192, 141, 260, 381, 281, 64, 423, 57, 324, 223, 132, 116, 67, 365, 311, 115, 210, 157, 225, 118, 41, 19, 421, 259, 219, 342, 397, 340, 29, 298, 159, 78, 107, 370, 15, 50, 383, 351, 238, 63, 361, 392, 98, 80, 198, 31

episode 3 start
episode batches: [166, 410, 328, 301, 39, 89, 272, 149, 180, 251, 72, 186, 202, 322, 82, 179, 116, 223, 378, 424, 391, 38, 371, 14, 354, 314, 83, 288, 415, 269, 305, 0, 318, 236, 93, 259, 394, 361, 326, 335, 205, 197, 165, 365, 307, 331, 227, 168, 384, 243, 124, 273, 13, 51, 12, 292, 225, 154, 101, 95, 201, 207, 20, 235, 402, 203, 60, 148, 351, 167, 109, 19, 137, 412, 377, 24, 115, 407, 398, 25, 388, 114, 329, 31, 106, 353, 63, 291, 163, 175, 184, 26, 393, 379, 319, 233, 153, 418, 278, 220, 297, 159, 399, 11, 249, 181, 289, 304, 241, 143, 244, 128, 37, 316, 252, 277, 417, 169, 237, 77, 226, 27, 263, 130, 296, 396, 108, 246, 348, 199, 62, 406, 387, 337, 229, 419, 330, 408, 357, 78, 334, 380, 369, 18, 254, 59, 381, 4, 52, 42, 174, 136, 145, 350, 113, 308, 355, 185, 6, 64, 172, 359, 321, 118, 260, 257, 158, 265, 392, 97, 231, 16, 420, 218, 142, 3, 261, 290, 132, 117, 332, 34, 162, 375, 53, 342, 338, 22, 411, 311, 421, 306, 152, 2, 383, 245, 135, 188, 212, 170, 425, 133, 35

episode 5 start
episode batches: [190, 356, 103, 21, 204, 155, 292, 106, 121, 32, 250, 85, 18, 62, 31, 131, 210, 418, 363, 387, 258, 274, 352, 33, 133, 22, 310, 229, 59, 388, 209, 296, 80, 265, 351, 5, 164, 276, 202, 308, 141, 268, 78, 371, 23, 263, 7, 19, 182, 332, 407, 163, 399, 114, 422, 251, 344, 398, 58, 342, 10, 194, 243, 218, 112, 52, 29, 336, 1, 364, 241, 44, 224, 41, 79, 334, 425, 149, 195, 234, 294, 291, 216, 199, 385, 365, 4, 279, 343, 3, 408, 376, 169, 313, 225, 98, 72, 132, 238, 0, 13, 396, 424, 184, 213, 222, 153, 137, 101, 280, 375, 68, 174, 338, 88, 123, 110, 135, 175, 207, 201, 309, 84, 25, 26, 177, 205, 27, 368, 145, 2, 48, 208, 298, 105, 67, 90, 47, 139, 189, 254, 54, 57, 314, 401, 391, 300, 81, 128, 340, 271, 206, 147, 70, 347, 69, 56, 360, 28, 108, 252, 240, 63, 282, 323, 143, 180, 34, 125, 24, 76, 30, 286, 185, 299, 415, 324, 12, 386, 283, 379, 378, 136, 152, 39, 157, 350, 273, 203, 322, 83, 61, 126, 413, 168, 99, 278, 107, 354, 302, 35, 417, 287, 307, 318, 316, 3

episode 7 start
episode batches: [174, 112, 336, 98, 156, 168, 325, 226, 364, 49, 17, 57, 397, 13, 249, 371, 6, 384, 108, 198, 140, 245, 368, 69, 41, 337, 117, 396, 414, 38, 428, 420, 239, 167, 297, 210, 265, 81, 139, 35, 44, 408, 77, 307, 47, 410, 403, 93, 411, 291, 90, 298, 257, 159, 229, 196, 310, 126, 85, 236, 360, 314, 118, 340, 113, 148, 419, 79, 422, 56, 279, 25, 190, 235, 427, 359, 227, 358, 393, 326, 344, 252, 207, 183, 290, 309, 243, 91, 115, 276, 114, 331, 188, 106, 170, 352, 153, 50, 201, 416, 327, 271, 330, 404, 4, 240, 356, 215, 354, 124, 103, 179, 208, 96, 282, 367, 192, 11, 426, 176, 33, 303, 42, 349, 129, 418, 348, 332, 248, 218, 36, 338, 122, 381, 166, 73, 217, 143, 20, 203, 181, 425, 104, 121, 221, 24, 197, 14, 186, 9, 230, 149, 262, 147, 293, 211, 138, 134, 313, 131, 238, 390, 275, 311, 0, 272, 391, 144, 92, 43, 280, 373, 23, 164, 12, 141, 369, 219, 195, 295, 52, 206, 385, 80, 162, 97, 308, 209, 412, 28, 31, 274, 123, 273, 7, 424, 189, 370, 388, 214, 342, 423, 127, 

episode 9 start
episode batches: [315, 340, 260, 100, 106, 155, 319, 334, 62, 210, 71, 129, 292, 131, 375, 176, 121, 143, 265, 335, 414, 184, 200, 114, 11, 20, 358, 261, 196, 41, 1, 374, 364, 362, 275, 336, 359, 160, 377, 348, 326, 224, 411, 320, 170, 371, 360, 409, 255, 425, 98, 95, 385, 211, 94, 234, 24, 190, 272, 209, 147, 156, 110, 53, 266, 306, 84, 9, 140, 311, 69, 174, 6, 214, 150, 238, 168, 221, 383, 167, 204, 246, 36, 299, 241, 56, 218, 263, 127, 166, 208, 199, 50, 119, 363, 421, 205, 324, 113, 229, 43, 0, 424, 399, 318, 365, 296, 163, 181, 154, 180, 18, 355, 342, 206, 40, 279, 142, 304, 108, 347, 117, 183, 216, 416, 45, 144, 239, 13, 86, 134, 128, 392, 59, 387, 187, 236, 76, 285, 194, 198, 125, 159, 202, 89, 269, 195, 28, 186, 403, 297, 284, 258, 171, 149, 317, 178, 323, 58, 331, 157, 354, 68, 21, 136, 337, 426, 386, 169, 308, 288, 357, 417, 192, 278, 161, 153, 35, 52, 60, 413, 303, 237, 123, 63, 172, 82, 228, 19, 173, 366, 133, 252, 27, 289, 410, 87, 191, 356, 66, 373, 314, 2

...

episode 277 start
episode batches: [243, 250, 222, 142, 138, 201, 259, 164, 194, 53, 254, 145, 322, 356, 375, 366, 336, 143, 112, 205, 330, 56, 200, 215, 110, 12, 52, 14, 5, 3, 107, 281, 125, 0, 418, 72, 384, 147, 86, 122, 8, 371, 295, 6, 186, 1, 178, 294, 195, 403, 282, 170, 238, 289, 171, 121, 396, 425, 416, 54, 400, 237, 365, 377, 233, 368, 279, 163, 240, 405, 261, 84, 36, 179, 148, 397, 188, 343, 16, 255, 266, 393, 99, 286, 117, 102, 299, 47, 291, 61, 373, 326, 345, 39, 239, 248, 402, 111, 167, 140, 304, 422, 168, 413, 158, 69, 29, 307, 153, 288, 152, 284, 42, 177, 73, 10, 334, 327, 382, 292, 323, 17, 132, 351, 131, 230, 49, 347, 411, 219, 312, 123, 423, 256, 340, 50, 249, 128, 410, 173, 93, 134, 313, 276, 94, 184, 155, 31, 130, 244, 37, 302, 280, 398, 95, 267, 369, 150, 216, 332, 339, 190, 296, 383, 234, 263, 372, 226, 218, 370, 229, 206, 262, 337, 409, 391, 415, 2, 217, 379, 76, 74, 275, 228, 260, 329, 428, 160, 66, 124, 172, 311, 57, 92, 360, 414, 23, 245, 387, 55, 324, 252, 62,

episode 279 start
episode batches: [364, 83, 405, 270, 331, 260, 421, 35, 358, 21, 166, 346, 107, 77, 33, 116, 216, 294, 369, 26, 69, 9, 139, 82, 385, 127, 196, 251, 316, 51, 169, 370, 132, 49, 261, 20, 203, 141, 281, 174, 39, 194, 228, 122, 58, 355, 178, 155, 114, 200, 400, 419, 391, 162, 425, 371, 362, 348, 66, 429, 222, 417, 181, 272, 301, 357, 390, 386, 8, 195, 63, 269, 306, 85, 143, 190, 257, 396, 220, 232, 359, 319, 3, 315, 420, 115, 31, 398, 282, 29, 276, 327, 295, 239, 341, 343, 102, 325, 372, 249, 236, 361, 380, 366, 258, 211, 330, 34, 52, 241, 317, 71, 125, 392, 103, 111, 198, 303, 57, 267, 288, 186, 160, 388, 148, 322, 214, 56, 246, 156, 25, 407, 238, 323, 265, 170, 297, 378, 227, 150, 308, 363, 142, 394, 157, 310, 379, 374, 177, 202, 50, 209, 93, 259, 109, 321, 189, 226, 256, 73, 354, 46, 345, 351, 218, 278, 161, 16, 126, 367, 285, 244, 23, 273, 389, 67, 90, 138, 27, 305, 134, 426, 409, 338, 41, 64, 81, 104, 151, 47, 32, 326, 217, 221, 4, 6, 291, 427, 210, 219, 40, 212, 382

episode 281 start
episode batches: [385, 276, 70, 312, 114, 24, 50, 33, 361, 53, 370, 153, 395, 420, 274, 76, 288, 357, 308, 239, 247, 259, 245, 107, 275, 162, 75, 255, 166, 89, 103, 313, 134, 326, 121, 35, 299, 222, 419, 233, 180, 280, 383, 130, 414, 303, 30, 143, 165, 328, 66, 391, 191, 139, 262, 69, 185, 87, 214, 330, 265, 410, 292, 388, 250, 112, 237, 142, 230, 63, 204, 279, 291, 285, 340, 324, 428, 137, 37, 71, 348, 411, 194, 36, 417, 365, 172, 302, 422, 224, 109, 140, 108, 252, 232, 398, 321, 167, 120, 290, 133, 62, 123, 85, 286, 150, 34, 401, 409, 289, 163, 293, 154, 393, 135, 147, 131, 282, 25, 244, 412, 345, 169, 56, 261, 355, 49, 314, 94, 211, 317, 246, 241, 28, 347, 251, 336, 2, 74, 429, 125, 64, 46, 171, 415, 187, 208, 351, 174, 209, 97, 353, 227, 400, 277, 183, 375, 319, 322, 407, 55, 15, 179, 39, 377, 270, 402, 248, 387, 205, 7, 297, 18, 271, 386, 77, 338, 176, 215, 9, 242, 320, 52, 157, 190, 269, 381, 80, 394, 138, 20, 234, 91, 418, 360, 155, 264, 81, 48, 344, 96, 339, 3

episode 283 start
episode batches: [179, 400, 398, 345, 1, 243, 6, 33, 109, 74, 39, 138, 229, 3, 263, 8, 429, 420, 100, 271, 348, 43, 339, 11, 107, 53, 85, 357, 298, 372, 299, 37, 369, 120, 227, 294, 22, 134, 16, 50, 60, 199, 244, 338, 259, 44, 408, 268, 128, 351, 13, 67, 315, 17, 226, 45, 4, 230, 312, 340, 300, 401, 317, 47, 54, 264, 419, 153, 157, 42, 382, 380, 83, 84, 248, 323, 273, 123, 289, 245, 385, 318, 198, 139, 349, 64, 135, 332, 140, 176, 89, 290, 342, 180, 147, 218, 127, 325, 327, 224, 297, 194, 324, 117, 91, 316, 277, 341, 303, 175, 397, 278, 225, 137, 223, 304, 328, 151, 69, 146, 61, 2, 364, 203, 210, 335, 124, 261, 88, 388, 178, 414, 144, 371, 254, 406, 68, 46, 36, 162, 29, 378, 95, 257, 149, 389, 187, 5, 212, 167, 356, 122, 231, 266, 217, 202, 301, 412, 93, 306, 181, 293, 51, 182, 106, 387, 370, 416, 361, 322, 367, 75, 291, 336, 252, 314, 403, 236, 152, 234, 329, 165, 283, 209, 319, 292, 108, 49, 333, 59, 402, 103, 207, 355, 376, 102, 195, 136, 394, 142, 30, 343, 166, 41

episode 285 start
episode batches: [103, 109, 415, 21, 110, 409, 265, 157, 140, 407, 65, 245, 232, 268, 91, 10, 213, 360, 302, 320, 235, 211, 204, 310, 332, 198, 52, 327, 117, 90, 296, 164, 86, 300, 17, 278, 118, 421, 127, 129, 405, 28, 377, 5, 66, 174, 361, 34, 12, 87, 347, 221, 416, 225, 0, 253, 330, 275, 114, 279, 144, 56, 269, 46, 333, 273, 31, 94, 195, 196, 197, 426, 192, 391, 328, 289, 390, 305, 226, 163, 20, 122, 290, 427, 372, 249, 291, 272, 236, 231, 53, 343, 201, 252, 64, 318, 325, 159, 247, 171, 382, 365, 424, 189, 351, 165, 352, 233, 334, 304, 317, 321, 150, 11, 363, 346, 132, 366, 293, 322, 345, 315, 297, 277, 208, 35, 156, 319, 175, 403, 375, 97, 411, 368, 364, 49, 194, 373, 199, 254, 258, 145, 153, 285, 355, 202, 119, 246, 89, 380, 120, 15, 238, 242, 287, 220, 178, 40, 227, 205, 50, 401, 314, 400, 329, 429, 84, 130, 126, 187, 379, 42, 294, 88, 203, 78, 108, 104, 250, 292, 143, 83, 60, 116, 362, 112, 215, 262, 105, 63, 68, 410, 260, 19, 367, 397, 41, 311, 276, 58, 80, 79,

In [ ]:
plt.subplots(figsize=(15, 9))
plt.plot(sliding_error_list)

In [ ]:
pd.DataFrame(Q).to_csv('Experiments_'+ str(experiment)+ '_Q.csv') 

In [ ]:
pd.DataFrame(Q)

In [ ]:
'Experiments_'+ str(experiment)+ '_Q.csv'